# 오토 라벨링 성능 평가 코드


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# auto labeling
auto = pd.read_csv("/content/drive/MyDrive/2023_연구학점/jiae/multilabeled_everytime0603_3.csv")

# manual labeling
manual1 = pd.read_excel("/content/drive/MyDrive/2023_연구학점/jiae/multilabeled_everytime2.xlsx")
manual2 = pd.read_csv("/content/drive/MyDrive/2023_연구학점/jiae/지희님 데이터 사용/지희님 라벨링 데이터.csv")

In [ ]:
auto_label = auto['label']

In [ ]:
auto_label

0              []
1              []
2              []
3              []
4          [4, 7]
          ...    
1193           []
1194          [9]
1195    [4, 6, 9]
1196          [9]
1197    [1, 2, 3]
Name: label, Length: 1198, dtype: object

In [ ]:
manual_label1 = manual1['labels']

In [ ]:
manual_label1

0        0
1        0
2      1,2
3        0
4      1,7
      ... 
995      0
996      7
997      0
998      9
999      0
Name: labels, Length: 1000, dtype: object

In [ ]:
manual_label2 = manual2['labels']

In [ ]:
manual_label2

0                  [2]
1         [2, 4, 5, 6]
2               [6, 9]
3      [1, 2, 5, 6, 7]
4            [1, 5, 6]
            ...       
193             [1, 5]
194                [1]
195                [5]
196       [1, 2, 6, 8]
197             [1, 6]
Name: labels, Length: 198, dtype: object

In [ ]:
manual_label = manual_label1.to_list()+manual_label2.to_list()

In [ ]:
auto_label= pd.DataFrame(auto_label)
auto_label = auto_label.rename(columns = {'label':'auto_label'})

manual_label= pd.DataFrame({'manual_label':manual_label})
#manual_label = manual_label.rename(columns = {'labels':'manual_label'})

In [ ]:
df_label = pd.concat([auto_label, manual_label], axis=1) # auto labeling, manual labeling 결과 dataframe 만들기
df_label

,auto_label,manual_label
0,[],0
1,[],0
2,[],"1,2"
3,[],0
4,"[4, 7]","1,7"
...,...,...
1193,[],"[1, 5]"
1194,[9],[1]
1195,"[4, 6, 9]",[5]
1196,[9],"[1, 2, 6, 8]"


In [ ]:
df_label.dropna(inplace=True)  # manual labeing이 안 되어 있는 부분 삭제
df_label.reset_index(drop=True, inplace=True)

In [ ]:
df_label

,auto_label,manual_label
0,[],0
1,[],0
2,[],"1,2"
3,[],0
4,"[4, 7]","1,7"
...,...,...
945,[],"[1, 5]"
946,[9],[1]
947,"[4, 6, 9]",[5]
948,[9],"[1, 2, 6, 8]"


In [ ]:
# auto labeling 결과와 manual labeling 결과 표현 형태 일치 시키기
df_label.auto_label = df_label.auto_label.apply(lambda x: list(map(int, x.strip('][').split(', '))) if x != '[]' else [0]) 
df_label['auto_label'][0]

[0]

In [ ]:
df_label.manual_label[:-198] = df_label.manual_label[:-198].apply(lambda x: x.strip() if type(x)==str else x)

In [ ]:
df_label.manual_label[:-198] = df_label.manual_label[:-198].apply(lambda x: list(map(int, x.split(','))) if type(x)==str else [x])

In [ ]:
df_label.manual_label[-198:] = df_label.manual_label[-198:].apply(lambda x: list(map(int, x.strip('][').split(', '))) if x != '[]' else [0])

In [ ]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   auto_label    950 non-null    object
 1   manual_label  950 non-null    object
dtypes: object(2)
memory usage: 15.0+ KB


In [ ]:
label_pred = df_label['auto_label']

In [ ]:
label_true = df_label['manual_label']

In [ ]:
df_label

,auto_label,manual_label
0,[0],[0]
1,[0],[0]
2,[0],"[1, 2]"
3,[0],[0]
4,"[4, 7]","[1, 7]"
...,...,...
945,[0],"[1, 5]"
946,[9],[1]
947,"[4, 6, 9]",[5]
948,[9],"[1, 2, 6, 8]"


In [ ]:
#False Positive Rate = FP/TN+FP 아닌데 맞다고 할 확률
def FPR(lb):
  tnfp = 0
  fp = 0
  lb= lb
  for i, label in enumerate(df_label['manual_label']):
    if lb not in label:
      tnfp+=1

      if lb in df_label['auto_label'][i]:
        fp += 1 
  return fp/tnfp

print(FPR(1))
print(FPR(2))
print(FPR(3))
print(FPR(4))
print(FPR(5))
print(FPR(6))
print(FPR(7))
print(FPR(8))
print(FPR(9))

0.04234527687296417
0.08108108108108109
0.12072649572649573
0.09081081081081081
0.04290822407628129
0.017857142857142856
0.16376306620209058
0.0249728555917481
0.17153748411689962


In [ ]:
label_pred[0]

[0]

In [ ]:
def onehot(label_list):
  onehot = [0 for _ in range(9)]
  for i in range(10):
    if i in label_list:
      onehot[i-1]=1
  return onehot


In [ ]:
labels = []
for l in label_pred:
  labels.append(onehot(l))

label_pred = np.array(labels)

In [ ]:
labels = []
for l in label_true:
  labels.append(onehot(l))

label_true = np.array(labels)

In [ ]:
label_pred

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
#Exact match ratio
MR = np.all(label_pred == label_true).mean()
MR

0.0

In [ ]:
def Accuracy(y_true, y_pred):
  temp = 0
  for i in range(y_true.shape[0]):
      temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
  return temp / y_true.shape[0]
  
Accuracy(label_true, label_pred)

0.508604040020263

In [ ]:
label_true.shape

(752, 9)

In [ ]:
def Hamming_Loss(y_true, y_pred):
  temp=0
  for i in range(y_true.shape[0]):
      temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
  return temp/(y_true.shape[0] * y_true.shape[1])
  
Hamming_Loss(label_true, label_pred)

0.15307328605200946

In [ ]:
def Recall(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_true[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
    return temp/ y_true.shape[0]
Recall(label_true, label_pred)

0.5620567375886523

In [ ]:
def Precision(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_pred[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
    return temp/ y_true.shape[0]
Precision(label_true, label_pred)

0.5485593971631205

In [ ]:
def F1Measure(y_true, y_pred):
  temp = 0
  for i in range(y_true.shape[0]):
      if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
          continue
      temp+= (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
  return temp/ y_true.shape[0]
  
print(F1Measure(label_true, label_pred))

0.5395864994934145


In [ ]:
import sklearn.metrics

print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(label_true, label_pred, normalize=True, sample_weight=None)))
#Exact Match Ratio: 0.25

print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(label_true, label_pred))) 
#Hamming loss: 0.4166666666666667

#"samples" applies only to multilabel problems. It does not calculate a per-class measure, instead calculating the metric over the true and predicted classes 
#for each sample in the evaluation data, and returning their (sample_weight-weighted) average.

print('Recall: {0}'.format(sklearn.metrics.precision_score(y_true=label_true, y_pred=label_pred, average='samples'))) 
#Recall: 0.375

print('Precision: {0}'.format(sklearn.metrics.recall_score(y_true=label_true, y_pred=label_pred, average='samples')))
#Precision: 0.5

print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_true=label_true, y_pred=label_pred, average='samples'))) 
#F1 Measure: 0.41666666666666663

Exact Match Ratio: 0.43351063829787234
Hamming loss: 0.15307328605200946
Recall: 0.5485593971631205
Precision: 0.5620567375886524
F1 Measure: 0.5395864994934143


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# 우울/비우울 잘 골랐는지
auto_zero = []
manual_zero = []
for label in df_label['auto_label']:
  if label==[0]:
    auto_zero.append(1)
  else:
    auto_zero.append(0)

for label in df_label['manual_label']:
  if label==[0]:
    manual_zero.append(1)
  else:
    manual_zero.append(0)

a = np.array(auto_zero)
m = np.array(manual_zero)

print('label 0 precision: ', np.sum(a)/len(m))


label 0 precision:  0.4348404255319149
